In [1]:
import itertools
import numpy as np
import pandas as pd
import random as r
import math

import matplotlib.pyplot as plot

In [2]:
I = [1, 2, 3] #set of items
J = [1, 2, 3] #set of departments/customers
T = list(range(1,4)) #four weeks/days...etc.
S = [1, 2, 3] #best (min) , expected, worst (max)

In [3]:
#model assumptions
safety_stock = [10,20,40]
beggining_stock = [20,40,80]
days_until_order_arrival = [-1,-1,-1] #where -1 means there is no current open orders

In [4]:
d = dict(zip(list(itertools.product(I, J, S)),
        [round(r.random()*1*7) for i in range(len(I)*len(J))] +
         [round(r.random()*4*7) for i in range(len(I)*len(J))] +
         [round(r.random()*10*7) for i in range(len(I)*len(J))] 
        )) #projected weekly demand in scenario
l = dict(zip(list(itertools.product(I, S)),
            [2,3,1] +
             [3,4,2] + 
             [3,4,3])) #expected lead time in weeks in scenario S (best, expected, worst)
#q = dict(zip(list(itertools.product(I, S)),
#        [0,0,0] +
#        [.02, .03, .01] + 
#        [.02, .03, .01])) #fraction of poor quality items in scenario S (best, expected, worst case)
o = [100,100,100] #ordering costs per order
h = [100,100,100] #holding costs per order

In [5]:
def eoq():
    
    global d
    global o
    global h
    
    eoq_dict = dict()
    
    for i in I:
        for s in S:
            demand_value = 0
            for j in J:
                demand_value += d[(i,j,s)]
            eoq_value = ((2*demand_value*o[i-1])/h[i-1])**(1/2)
            eoq_dict[(i,s)] = math.ceil(eoq_value)
            
    return(eoq_dict)

In [6]:
def reorder_point(safety_stock):
    
    global d
    global l
    
    reorder_point_dict = dict()
    
    for i in I:
        for s in S:
            demand_value = 0
            for j in J:
                demand_value += d[(i,j,s)]
            lead_time_value = l[(i,s)]
            reorder_point_value = lead_time_value*demand_value + safety_stock[i-1]
            reorder_point_dict[(i,s)] = reorder_point_value
            
    return(reorder_point_dict)

In [7]:
def sim(safety_stock, beggining_stock, days_until_order_arrival):
    eoq_dict = eoq()
    reorder_point_dict = reorder_point(safety_stock)
    
    unsatisfied_demand_dict = dict()
    
    for s_planned in S:
        for s_actual in S:
            current_stock_array = beggining_stock
            days_until_order_arrival = days_until_order_arrival
            for t in T:
                for j in J:
                    for i in I:
                        #distribution
                        demand_value = d[(i,j,s_actual)]
                        satisfied_demand_value = min(current_stock_array[i-1], demand_value)
                        unsatisfied_demand_value = demand_value - satisfied_demand_value
                        unsatisfied_demand_dict[(i,j,t,s_planned,s_actual)] = unsatisfied_demand_value
                        
                        current_stock_array[i-1] = current_stock_array[i-1] - satisfied_demand_value
                        
            #at the end of the day check if anything needs to be reordered (has hit reorder point)
            if current_stock_array[i-1] < reorder_point_dict[(i, s_planned)]:
                if days_until_order_arrival[i-1] < 0: #no open order
                    days_until_order_arrival[i-1] = l[(i,s_actual)]

            #at the end of the day available stock that has arrived
            if days_until_order_arrival[i-1] == 0:
                current_stock_array[i-1] += eoq_dict[(i,s_planned)]
                
            #update day until next arrival
            days_until_order_arrival = [x - 1 for x in days_until_order_arrival]
    
    return(unsatisfied_demand_dict)

In [8]:
unsatisfied_demand_dict = sim(safety_stock = safety_stock, beggining_stock = beggining_stock, days_until_order_arrival = days_until_order_arrival)

In [41]:
columns = ['planned_scenario', 'actual_scenario', 'time', 'department', 'item', 'unsatisfied_demand']

df = pd.DataFrame(columns = columns)
    
for s_planned in S:
    for s_actual in S:
        for t in T:
            for j in J:
                for i in I:
                    row = [s_planned, s_actual, t, j, i, unsatisfied_demand_dict[(i,j,t,s_planned, s_actual)]]
                    df_temp = pd.DataFrame(np.array(row).reshape(-1,len(row)), columns = columns)
                    df = df.append(df_temp)

In [45]:
df[['planned_scenario', 'actual_scenario', 'item', 'unsatisfied_demand']].groupby(['planned_scenario', 'actual_scenario', 'item']).sum()

unsatisfied_demand
planned_scenario actual_scenario item                    
1                1               1                     25
                                 2                     38
                                 3                    367
                 2               1                     30
                                 2                     87
                                 3                    249
                 3               1                     42
                                 2                    123
                                 3                    396
2                1               1                     45
                                 2                     78
                                 3                    447
                 2               1                     30
                                 2                     87
                                 3                    236
                 3               1                     42
                                 2                    123
                                 3                    396
3                1               1                     45
                                 2                     78
                                 3                    447
                 2               1                     30
                                 2                     87
                                 3                    249
                 3               1                     42
                                 2                    123
                                 3                    396

In [ ]:
df=pd.DataFrame({'x': range(1,11), 'y1': np.random.randn(10), 'y2': np.random.randn(10)+range(1,11), 'y3': np.random.randn(10)+range(11,21) })
 
# multiple line plot
plt.plot( 'x', 'y1', data=df, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)
plt.plot( 'x', 'y2', data=df, marker='', color='olive', linewidth=2)
plt.plot( 'x', 'y3', data=df, marker='', color='olive', linewidth=2, linestyle='dashed', label="toto")

In [11]:
unsatisfied_demand_dict

{(1, 1, 1, 1, 1): 0,
 (2, 1, 1, 1, 1): 0,
 (3, 1, 1, 1, 1): 0,
 (1, 2, 1, 1, 1): 0,
 (2, 2, 1, 1, 1): 0,
 (3, 2, 1, 1, 1): 38,
 (1, 3, 1, 1, 1): 0,
 (2, 3, 1, 1, 1): 0,
 (3, 3, 1, 1, 1): 31,
 (1, 1, 2, 1, 1): 1,
 (2, 1, 2, 1, 1): 0,
 (3, 1, 2, 1, 1): 50,
 (1, 2, 2, 1, 1): 7,
 (2, 2, 2, 1, 1): 2,
 (3, 2, 2, 1, 1): 68,
 (1, 3, 2, 1, 1): 2,
 (2, 3, 2, 1, 1): 10,
 (3, 3, 2, 1, 1): 31,
 (1, 1, 3, 1, 1): 6,
 (2, 1, 3, 1, 1): 3,
 (3, 1, 3, 1, 1): 50,
 (1, 2, 3, 1, 1): 7,
 (2, 2, 3, 1, 1): 13,
 (3, 2, 3, 1, 1): 68,
 (1, 3, 3, 1, 1): 2,
 (2, 3, 3, 1, 1): 10,
 (3, 3, 3, 1, 1): 31,
 (1, 1, 1, 1, 2): 2,
 (2, 1, 1, 1, 2): 3,
 (3, 1, 1, 1, 2): 35,
 (1, 2, 1, 1, 2): 1,
 (2, 2, 1, 1, 2): 7,
 (3, 2, 1, 1, 2): 19,
 (1, 3, 1, 1, 2): 7,
 (2, 3, 1, 1, 2): 19,
 (3, 3, 1, 1, 2): 29,
 (1, 1, 2, 1, 2): 2,
 (2, 1, 2, 1, 2): 3,
 (3, 1, 2, 1, 2): 35,
 (1, 2, 2, 1, 2): 1,
 (2, 2, 2, 1, 2): 7,
 (3, 2, 2, 1, 2): 19,
 (1, 3, 2, 1, 2): 7,
 (2, 3, 2, 1, 2): 19,
 (3, 3, 2, 1, 2): 29,
 (1, 1, 3, 1, 2): 2,
 (2, 1, 3, 1, 2

In [12]:
pd.DataFrame.from_dict(unsatisfied_demand_dict)

ValueError: If using all scalar values, you must pass an index